# Graph Theory Class assignment

## Try me
[![Open In Colab](../../_static/colabs_badge.png)](https://colab.research.google.com/github/ffraile/operations-research-notebooks/blob/main/docs/source/MIP/tutorials/Traveling%20Salesman%20Problem%20Example.ipynb)[![Binder](../../_static/binder_badge.png)](https://mybinder.org/v2/gh/ffraile/operations-research-notebooks/main?labpath=docs%2Fsource%2FMIP%2Ftutorials%2FTraveling%20Salesman%20Problem%20Example.ipynb)

## Requirements

You need to install Networkx in your runtime, so make sure you run this script if not already installed. 

Note that the installation line includes the *extra* option to install additional packages which are useful to work with
Networkx.

In [ ]:
!pip install networkx[default]
!pip install ipython
!pip install pandas

Additionally, you need to install osmnx and Folium:


In [ ]:
!apt-get -qq install -y libspatialindex-dev
!pip install osmnx
!pip install folium

Finally, you need to install scikit-learn to use the Traveling Salesman Problem algorithm. Note that this is already pre-installed in Google Colabs.

In [ ]:
!pip install scikit-learn

#### Create a map
Create a network from a map using the ```graph_from_bbox``` function:

- ```graph_from_bbox```: Creates a graph within four points of a bounding box, specifying the ```north``` northern 
latitude of the bounding box, the ```south``` southern latitude of bounding box, the ```east``` eastern longitude of 
the bounding box and the ```west``` western longitude of the bounding box.

Use Google Maps to get two markers around the area of interest of your assignment, it can be your neighbourhood, or anywhere in the world, but EDEM. Then modify the coordinates of the map in the following snippet:

In [4]:
import networkx as nx
import osmnx as ox
import pandas as pd
# get two markers from Google Maps and modify the coordinates around EDEM
# e.g. with two markers around EDEM: Latitude, Longitude
# North-west marker
# 39.468370277652426, -0.33528989341188187
# 39.47953561643489, -0.38186895348265193
# South east marker
# 39.45963084307057, -0.3184406656568226
# 39.47953561643489, -0.38186895348265193
# 39.469446, -0.365671


G_nx = ox.graph_from_bbox(north=39.468370277652426, 
                          south=39.45963084307057, 
                          east=-0.3184406656568226, 
                          west=-0.33528989341188187)

#### Rendering the map
The function plot_graph_folium uses [Folium](https://python-visualization.github.io/folium/) to render the map in an 
interactive map:

In [5]:
m1 = ox.plot_graph_folium(G_nx)
m1

#### distance.nearest_node
When you work on a graph, you could click on a point in the map that is not necessarily an edge in the graph, so
to be able to process its coordinates, you first need to find the node in the graph that is closest to the point. You can use the  [nearest_node](https://osmnx.readthedocs.io/en/stable/osmnx.html#osmnx.distance.nearest_nodes) function for this purpose, it returns the id of the node that is closest to the point you pass as an argument so
that you can later use the node in the algorithms. Let see with an example, using the coordinates of EDEM extracted from Google Maps:

In [6]:
edem_latitude = 39.46211739713285
edem_longitude = -0.3288013882525529
nearest_node = ox.distance.nearest_nodes(G_nx, X=edem_longitude, Y=edem_latitude)
nearest_node

6629446599

### Exploring the data available in the graph
The function [to_pandas_edgelist](https://networkx.org/documentation/stable/reference/generated/networkx.convert_matrix.to_pandas_edgelist.html) returns a dataframe with the edges and their data. We can use this function to explore the data available in the graph:



In [7]:
# Get the edges and their data and put the data in a dataframe
nx.to_pandas_edgelist(G_nx)


,source,target,service,reversed,osmid,highway,maxspeed,length,access,lanes,oneway,junction,name,geometry
0,88356838,1800312429,NaN,False,652764215,residential,NaN,34.784,NaN,NaN,True,NaN,Plaça de les Hores,"LINESTRING (-0.332837 39.4599649, -0.332748 39..."
1,88356838,1339801172,NaN,False,806146193,secondary,NaN,17.184,NaN,NaN,True,NaN,Carrer del Doctor Josep Juan Dòmine,"LINESTRING (-0.332837 39.4599649, -0.3328162 3..."
2,88428374,5499046929,NaN,False,115600553,residential,NaN,11.675,NaN,NaN,True,NaN,Plaça de la Setmana Santa Marinera,"LINESTRING (-0.3302816 39.4627739, -0.3302036 ..."
3,88428374,10139283393,NaN,False,115600559,residential,NaN,6.991,NaN,NaN,True,NaN,Carrer del Doctor Josep Juan Dòmine,"LINESTRING (-0.3302816 39.4627739, -0.3302669 ..."
4,88428405,6338241732,NaN,False,13960791,secondary,NaN,299.907,NaN,NaN,True,NaN,Carrer del Doctor Josep Juan Dòmine,"LINESTRING (-0.3302833 39.4624871, -0.3303942 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1479,10285307342,10285307326,NaN,False,1111546975,footway,NaN,10.080,NaN,NaN,False,NaN,NaN,NaN
1480,10285307342,10285307341,NaN,True,1111546975,footway,NaN,31.438,NaN,NaN,False,NaN,NaN,NaN
1481,10285307342,10285307341,NaN,False,1111546974,footway,NaN,59.922,NaN,NaN,False,NaN,NaN,"LINESTRING (-0.3239924 39.4636226, -0.3241564 ..."
1482,10285307342,10285307341,NaN,True,1111546974,footway,NaN,57.854,NaN,NaN,False,NaN,NaN,"LINESTRING (-0.3239924 39.4636226, -0.3238383 ..."




### Traveling salesman
Now, we want to calculate the optimal route to visit a set of points in the map solving the traveling salesman problem.
The function [traveling_salesman_problem](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.approximation.traveling_salesman.traveling_salesman_problem.html) solves this problem in any graph. As you can read in the documentation of this function, we need to specify which attribute of the edges in the graph specifies the weight used to calculate distances in the TSP problem.

By looking at the dataframe, we can tell that the attribute is called **length**.

#### Select points
Now you are ready to select different points in the map and use them in the TSP problem. Use again Google Maps to select some points (at least 4) in the map and add the latitudes and longitudes in the following cell. Remember you need to get the nearest nodes for all of them. The following example calculates the TSP between EDEM and a nearby museum

In [8]:
latitudes = (39.46211739713285, 39.46329739674903)
longitudes = (-0.3288013882525529, -0.33194815644130443)
nearest_nodes = ox.distance.nearest_nodes(G_nx, X=longitudes, Y=latitudes)
print(nearest_nodes)
route = nx.approximation.traveling_salesman_problem(G_nx.to_undirected(), nodes=nearest_nodes, weight="length")
ox.plot_route_folium(G_nx, route, route_map=m1, weight=7, color='black')

[6629446599, 430152412]
